# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [63]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [23]:
weather_data = pd.read_csv("../WeatherPy/output_data/cities.csv", encoding = "utf-8" )
weather_data = weather_data.rename(columns = {"Longtidue":"Longitude"})
weather_data.head()

,Name,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,Touros,100,BR,1584146424,100,-5.20,-35.46,75.20,10.60
1,Port Alfred,26,ZA,1584146424,97,-33.59,26.89,69.01,4.00
2,Dabakala,26,CI,1584146424,67,8.37,-4.43,81.41,5.93
3,Jumla,23,NP,1584146424,65,29.27,82.18,25.75,7.40
4,Kapaa,90,US,1584146143,64,22.08,-159.32,77.00,12.75


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [66]:
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = weather_data[["Latitude", "Longitude"]].astype(float)
humidity = weather_data["Humidity"].astype(float)

In [67]:

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [77]:
ideal_weather = weather_data.loc[(weather_data['Max Temp'] < 85)]
ideal_weather = ideal_weather.loc[(weather_data['Max Temp'] > 70)]
ideal_weather = ideal_weather.loc[(weather_data['Wind Speed'] < 10)]
ideal_weather = ideal_weather.loc[weather_data['Cloudiness'] < 10 ]
 
hotel_df = ideal_weather.dropna().reset_index(drop=True)
print(len(hotel_df))

22


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [78]:
hotel_df["Hotel Name"] = ""
hotel_df.head()

,Name,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Hotel Name
0,Hilton Head,1,US,1584146370,77,32.22,-80.75,73.99,5.82,
1,Lázaro Cárdenas,0,MX,1584146429,71,17.96,-102.20,79.93,7.23,
2,Latung,4,PH,1584146429,70,5.50,120.88,81.10,3.56,
3,Arraial do Cabo,0,BR,1584146432,94,-22.97,-42.03,77.00,5.82,
4,Piopio,0,NZ,1584146444,38,-38.47,175.02,77.00,4.00,


In [80]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


params = {
        "radius": 5000,
        "key": g_key,
        "keyword" : "hotel"
    }
for index, row in hotel_df.iterrows():
    lat = row['Latitude']
    lng = row['Longitude']
    
    params['location'] = f"{lat},{lng}"
    
    response = requests.get(url,params).json()
    
    try:
        print(f"Found the hotel {response['results'][0]['name']}. Adding to list")
        hotel_df.loc[index, "Hotel Name"] =(response['results'][0]['name'])
    except:
        print("Hotel not found")
        hotel_df.loc[index,"Hotel Name"] = "No Nearby Hotel Found"
hotel_df

Found the hotel Days Inn by Wyndham Hilton Head. Adding to list
Found the hotel City Express Lázaro Cárdenas. Adding to list
Hotel not found
Found the hotel Orlanova Hotel. Adding to list
Found the hotel Owls Nest Motel. Adding to list
Found the hotel Hospedaje Claudia. Adding to list
Found the hotel Sails Port Macquarie by Rydges. Adding to list
Found the hotel Premier Hotels & Resorts. Adding to list
Found the hotel Baths Motel. Adding to list
Found the hotel The Westin Resort & Spa, Puerto Vallarta. Adding to list
Found the hotel DoubleTree by Hilton Hotel Jacksonville Riverfront. Adding to list
Found the hotel The Pade Hotel. Adding to list
Found the hotel Sri laxmi lodge. Adding to list
Found the hotel OYO 2598 Hotel Chocolate. Adding to list
Found the hotel Pousada Kactus. Adding to list
Hotel not found
Hotel not found
Hotel not found
Found the hotel Watsons Cottage. Adding to list
Found the hotel Los Inmigrantes Hotel y Resto Bar. Adding to list
Found the hotel Frontier Hotel. A

,Name,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Hotel Name
0,Hilton Head,1,US,1584146370,77,32.22,-80.75,73.99,5.82,Days Inn by Wyndham Hilton Head
1,Lázaro Cárdenas,0,MX,1584146429,71,17.96,-102.20,79.93,7.23,City Express Lázaro Cárdenas
2,Latung,4,PH,1584146429,70,5.50,120.88,81.10,3.56,No Nearby Hotel Found
3,Arraial do Cabo,0,BR,1584146432,94,-22.97,-42.03,77.00,5.82,Orlanova Hotel
4,Piopio,0,NZ,1584146444,38,-38.47,175.02,77.00,4.00,Owls Nest Motel
5,Bonanza,0,NI,1584146444,67,14.03,-84.59,74.98,2.13,Hospedaje Claudia
6,Port Macquarie,3,AU,1584146445,64,-31.43,152.92,80.01,9.17,Sails Port Macquarie by Rydges
7,East London,2,ZA,1584146450,77,-33.02,27.91,75.34,1.25,Premier Hotels & Resorts
8,Moree,0,AU,1584146452,53,-29.47,149.85,75.20,4.70,Baths Motel
9,Ixtapa,5,MX,1584146457,65,20.70,-105.20,78.80,9.17,"The Westin Resort & Spa, Puerto Vallarta"


In [81]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations_h = hotel_df[["Latitude", "Longitude"]]

In [82]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations_h, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=[x for x in hotel_info])

fig = gmaps.figure()
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
    
# Display Map
fig

Figure(layout=FigureLayout(height='420px'))